In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm import tqdm

In [81]:
class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 32, (3, 3), (1, 1), (1, 1)) 
    self.conv2 = nn.Conv2d(32, 64, (3, 3), (1, 1), (1, 1))
    self.conv3 = nn.Conv2d(64, 128, (3, 3), (1, 1), (1, 1))


    self.fc1 = nn.Linear(128*8*8 ,128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, kernel_size=(2, 2))

    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, kernel_size=(2, 2))

    x = F.relu(self.conv3(x))
    x = F.max_pool2d(x, kernel_size=(2, 2))
    x = torch.flatten(x, start_dim=1)

    x = self.fc1(x)
    x = self.fc2(x)
    output = torch.softmax(x, dim=1)

    return output
      

In [82]:
!pip install wandb

In [83]:
import wandb
wandb.init(project="Persian Mnist torch")

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Model().to(device)
# wandb.watch(model)

In [85]:
batch_size = 64
epochs = 20
config = wandb.config
config.learning_rate = 0.001


In [86]:
from torchvision.transforms.transforms import Resize
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.Resize((70, 70)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

dataset = torchvision.datasets.ImageFolder(root="/content/drive/MyDrive/MNIST_persian", transform = transform)
train_data = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [87]:
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
loss_function = nn.CrossEntropyLoss()


In [88]:

def calc_acc(preds, labels):
  _, pred_max = torch.max(preds, 1)
  acc = torch.sum(pred_max == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [89]:
model.train()

for epoch in range(epochs):
  train_loss = 0.0
  train_acc = 0.0
  for images, labels in tqdm(train_data):

    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    preds = model(images)

    loss = loss_function(preds, labels)
    loss.backward()

    optimizer.step()

    train_loss += loss
    train_acc += calc_acc(preds, labels)

  total_loss = train_loss / len(train_data)
  total_acc = train_acc / len(train_data)
  print(f"Epoch: {epoch+1}, Loss: {total_loss}, Accuracy: {total_acc}")

  wandb.log({'epochs':  epoch + 1,'loss': total_loss,'acc': total_acc})

100%|██████████| 19/19 [00:03<00:00,  5.04it/s]


Epoch: 1, Loss: 2.304955244064331, Accuracy: 0.1156798245614035


100%|██████████| 19/19 [00:05<00:00,  3.39it/s]


Epoch: 2, Loss: 2.0693411827087402, Accuracy: 0.42790570175438597


100%|██████████| 19/19 [00:05<00:00,  3.73it/s]


...

Epoch: 18, Loss: 1.525356650352478, Accuracy: 0.9385964912280701


100%|██████████| 19/19 [00:03<00:00,  5.77it/s]


Epoch: 19, Loss: 1.5113000869750977, Accuracy: 0.9575109649122806


100%|██████████| 19/19 [00:03<00:00,  5.51it/s]

Epoch: 20, Loss: 1.5002306699752808, Accuracy: 0.9654605263157894


In [90]:
torch.save(model.state_dict(), "mnist_p.pth")